In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import time
import datetime
import nbformat
from scipy import stats

In [2]:
#Load data
trips_df = pd.read_csv('https://sds-aau.github.io/SDS-master/M1/data/trips.csv', index_col=0)
people_df = pd.read_csv('https://sds-aau.github.io/SDS-master/M1/data/people.csv', index_col = 0)
country_df = pd.read_csv('https://sds-aau.github.io/SDS-master/M1/data/countrylist.csv', index_col = 0)

In [3]:
## check rows in col that are na
#trips_df[trips_df['date_end'].isna()]
## example of rows to see if they changed
#trips_df.loc[[638,640]]
trips_df

,username,country,country_code,country_slug,date_end,date_start,latitude,longitude,place,place_slug
0,@lewellenmichael,Mexico,MX,mexico,2018-06-15,2018-06-04,21,-101,Guanajuato,mexico
1,@lewellenmichael,Mexico,MX,mexico,2018-06-03,2018-05-31,19,-99,Mexico City,mexico-city-mexico
2,@lewellenmichael,Mexico,MX,mexico,2017-11-05,2017-11-01,21,-86,Cancun,cancun-mexico
3,@lewellenmichael,Jordan,JO,jordan,2017-08-07,2017-07-24,31,35,Amman,amman-jordan
4,@waylandchin,China,CN,china,2017-03-18,2017-02-17,40,122,Yingkou,china
...,...,...,...,...,...,...,...,...,...,...
46505,@antonioc,Cuba,CU,cuba,2016-05-25,2016-05-19,21,-77,Cuba,cuba
46506,@antonioc,Costa Rica,CR,costa-rica,2016-05-18,2016-04-28,9,-83,Costa Rica,costa-rica
46507,@antonioc,United States,US,united-states,2016-04-26,2016-04-04,39,-75,Philadelphia,philadelphia-pa-united-states
46508,@jrearden,United States,US,united-states,2018-05-19,2018-05-12,30,-86,Destin,united-states


In [4]:
# when checking type of column ==> type object - bc numerous value types
trips_df['date_end'].dtype
# if checking which columns contain nan value shows date_end has nan
trips_df.isna().any()

username        False
country          True
country_code     True
country_slug     True
date_end         True
date_start      False
latitude        False
longitude       False
place           False
place_slug       True
dtype: bool

In [5]:
#change cols to datetime format and coerce so NaT is added for na values
date_cols = [col for col in trips_df.columns if col.startswith('date')]
for col in date_cols:
    trips_df[col]= pd.to_datetime(trips_df[col],format='%Y-%m-%d', errors='coerce')

In [6]:
#check type of data after casting
trips_df['date_start'].dtype

dtype('<M8[ns]')

In [7]:
#get trip duration 
trips_df['trip_duration'] = (trips_df['date_end'] - trips_df['date_start']).dt.days


In [8]:
#remove all trips with negative value
trips_df = trips_df[trips_df['trip_duration'] >=1]

In [9]:
trips_df['trip_duration'].min()

1.0

In [10]:
trips_df['trip_duration'].describe()
# bc mean is sensitive to ouliers and max is74633, while mean is so small (58) ==> outliers


count    44717.000000
mean        53.064584
std        660.414569
min          1.000000
25%          3.000000
50%          7.000000
75%         25.000000
max      74633.000000
Name: trip_duration, dtype: float64

In [11]:
#check how na's are handled
trips_df[trips_df['trip_duration'].isna()]

,username,country,country_code,country_slug,date_end,date_start,latitude,longitude,place,place_slug,trip_duration


In [12]:
# check if normal distribution -> if not ==> can't use z score to find it; https://careerfoundry.com/en/blog/data-analytics/how-to-find-outliers/

fig_hst = px.histogram(trips_df, x='trip_duration')

fig_hst.show()

In [13]:
# this is how i found the 97 qqantile value and remove all values over
value_qantile_x = np.quantile(trips_df['trip_duration'],0.97)

In [14]:
#remove all from uper limit
trips_df=trips_df[trips_df['trip_duration']<= value_qantile_x]
trips_df.head(5)

,username,country,country_code,country_slug,date_end,date_start,latitude,longitude,place,place_slug,trip_duration
0,@lewellenmichael,Mexico,MX,mexico,2018-06-15,2018-06-04,21,-101,Guanajuato,mexico,11.0
1,@lewellenmichael,Mexico,MX,mexico,2018-06-03,2018-05-31,19,-99,Mexico City,mexico-city-mexico,3.0
2,@lewellenmichael,Mexico,MX,mexico,2017-11-05,2017-11-01,21,-86,Cancun,cancun-mexico,4.0
3,@lewellenmichael,Jordan,JO,jordan,2017-08-07,2017-07-24,31,35,Amman,amman-jordan,14.0
4,@waylandchin,China,CN,china,2017-03-18,2017-02-17,40,122,Yingkou,china,29.0


In [15]:
country_df.index.rename('country_code', inplace=True)
country_df

,region,sub_region
country_code,,
AF,Asia,Southern Asia
AX,Europe,Northern Europe
AL,Europe,Southern Europe
DZ,Africa,Northern Africa
AS,Oceania,Polynesia
...,...,...
WF,Oceania,Polynesia
EH,Africa,Northern Africa
YE,Asia,Western Asia


In [16]:
#merge country df with trip df based on country code
trips_country_df = pd.merge(trips_df, country_df, on='country_code')

In [17]:
trips_country_df.set_index('date_start',inplace=True)

In [18]:
trips_country_df

,username,country,country_code,country_slug,date_end,latitude,longitude,place,place_slug,trip_duration,region,sub_region
date_start,,,,,,,,,,,,
2018-06-04,@lewellenmichael,Mexico,MX,mexico,2018-06-15,21,-101,Guanajuato,mexico,11.0,Americas,Latin America and the Caribbean
2018-05-31,@lewellenmichael,Mexico,MX,mexico,2018-06-03,19,-99,Mexico City,mexico-city-mexico,3.0,Americas,Latin America and the Caribbean
2017-11-01,@lewellenmichael,Mexico,MX,mexico,2017-11-05,21,-86,Cancun,cancun-mexico,4.0,Americas,Latin America and the Caribbean
2017-09-10,@jtompl,Mexico,MX,mexico,2017-09-25,19,-99,Mexico City,mexico-city-mexico,15.0,Americas,Latin America and the Caribbean
2017-09-01,@jtompl,Mexico,MX,mexico,2017-09-10,20,-87,Tulum,tulum-mexico,9.0,Americas,Latin America and the Caribbean
...,...,...,...,...,...,...,...,...,...,...,...,...
2017-05-16,@shadi,New Caledonia,NC,new-caledonia,2017-05-25,-22,166,Nouméa,noumea-new-caledonia,9.0,Oceania,Melanesia
2017-11-28,@oceanfiredance,British Virgin Islands,VG,british-virgin-islands,2017-12-15,18,-64,Road Town,road-town-british-virgin-islands,17.0,Americas,Latin America and the Caribbean
2017-08-09,@bneiluj,Papua New Guinea,PG,papua-new-guinea,2017-08-21,-6,143,Papua New Guinea,papua-new-guinea,12.0,Oceania,Melanesia


In [19]:
trips_country_df.index

DatetimeIndex(['2018-06-04', '2018-05-31', '2017-11-01', '2017-09-10',
               '2017-09-01', '2018-04-05', '2017-11-02', '2017-08-02',
               '2017-01-24', '2016-09-11',
               ...
               '2012-06-01', '2009-07-01', '2015-11-29', '2015-03-31',
               '2015-02-06', '2017-05-16', '2017-11-28', '2017-08-09',
               '2017-02-20', '2017-02-20'],
              dtype='datetime64[ns]', name='date_start', length=41637, freq=None)

In [20]:
# fill in nan with false
people_df.fillna(False,inplace=True)

In [21]:
#check education level values
people_df['education_raw'].unique()

array(["High School, Bachelor's Degree", False,
       "High School, Bachelor's Degree, Master's Degree",
       "Master's Degree", "Bachelor's Degree", 'High School',
       "Bachelor's Degree, Master's Degree"], dtype=object)

In [22]:
def get_max_str(lst):
    for i in range(0, len(lst)):
            if type(lst[i]) == type(True):
                lst[i] = str(lst[i])
    max_list = max(lst, key=len).split(',')

    for i in range(0, len(max_list)):
        max_list[i] = max_list[i].strip()

    return max_list


def split_column(df_to_change,df_column, column_to_split):

                  
    for row in range(len(df_to_change)):#[df_column]:
        for string_tocheck in column_to_split:
            col_name = string_tocheck.replace("'", "" ).replace(' ', '_')
                        
            if string_tocheck in str(df_to_change.loc[row, df_column]):
                
                df_to_change.loc[row,col_name]= True
            else: 
                df_to_change.loc[row,col_name] = False 
            
    
    return  df_to_change
                    

In [23]:

column_to_split_education = get_max_str(people_df['education_raw'].unique())
people_df = split_column(people_df,'education_raw',column_to_split_education)

In [24]:
column_to_split_startup = ['Startup Founder']
people_df = split_column(people_df,'work_raw',column_to_split_startup)

In [25]:
people_df.head(5)

,username,followers,following,work_raw,education_raw,High_School,Bachelors_Degree,Masters_Degree,Startup_Founder
0,@lewellenmichael,1,2,"Software Dev, Startup Founder, Finance, Crypto...","High School, Bachelor's Degree",True,True,False,True
1,@waylandchin,0,2,False,False,False,False,False,False
2,@karan,2,1,False,False,False,False,False,False
3,@skaboss217,0,1,False,False,False,False,False,False
4,@apwn,17,426,Web Dev,False,False,False,False,False


In [26]:
#see how many have high-school diploma
count_highschool_diploma = len(people_df[(people_df['High_School']==True)])
count_highschool_diploma

130

In [27]:
startupfounders_msc_diploma = len(people_df[(people_df['Startup_Founder']==True) & (people_df['Masters_Degree']==True)])
startupfounders_msc_diploma

53

In [28]:
# comparing people who have no higher education , should i have inclueded the false for highschool?

people_df[(people_df['Bachelors_Degree']==False) & (people_df['Masters_Degree']==False) & (people_df['High_School']==True)]

people_df.loc[83,:]

username                                       @jayphen
followers                                            13
following                                            10
work_raw            Web Dev, Software Dev, UI/UX Design
education_raw                               High School
High_School                                        True
Bachelors_Degree                                  False
Masters_Degree                                    False
Startup_Founder                                   False
Name: 83, dtype: object

In [29]:
people_df.loc[218,:]

username                                                    @lonnylot
followers                                                           3
following                                                           7
work_raw            Web Dev, Software Dev, Startup Founder, Crypto...
education_raw                                             High School
High_School                                                      True
Bachelors_Degree                                                False
Masters_Degree                                                  False
Startup_Founder                                                  True
Name: 218, dtype: object

In [30]:
msc_df = people_df[(people_df['Masters_Degree']==True)]
msc_max_followers = msc_df[msc_df['followers'] == msc_df['followers'].max()]
msc_max_followers 


,username,followers,following,work_raw,education_raw,High_School,Bachelors_Degree,Masters_Degree,Startup_Founder
2043,@levelsio,2182,353,"Software Dev, Startup Founder, Creative","High School, Bachelor's Degree, Master's Degree",True,True,True,True


In [31]:
#POTENTIAL FOR MORE

msc_max_followers_trips = trips_df[trips_df['username'] == '@levelsio']
msc_max_followers_trips

,username,country,country_code,country_slug,date_end,date_start,latitude,longitude,place,place_slug,trip_duration
27404,@levelsio,United Kingdom,UK,united-kingdom,2018-10-14,2018-09-14,51,0,London,london-united-kingdom,30.0
27405,@levelsio,Indonesia,ID,indonesia,2018-09-14,2018-08-16,-8,115,Canggu,canggu-bali-indonesia,29.0
27406,@levelsio,Thailand,TH,thailand,2018-08-16,2018-07-17,13,100,Bangkok,bangkok-thailand,30.0
27407,@levelsio,South Korea,KR,south-korea,2018-07-17,2018-06-25,37,126,Seoul,seoul-south-korea,22.0
27408,@levelsio,South Korea,KR,south-korea,2018-06-25,2018-06-22,35,128,Busan,busan-south-korea,3.0
...,...,...,...,...,...,...,...,...,...,...,...
27638,@levelsio,Netherlands,NL,netherlands,2000-07-14,2000-01-12,51,5,Nijmegen,nijmegen-netherlands,184.0
27639,@levelsio,Netherlands,NL,netherlands,2000-07-14,2000-01-11,51,5,Nijmegen,nijmegen-netherlands,185.0
27641,@levelsio,Greece,GR,greece,1995-07-31,1995-07-16,38,20,Lefkada,greece,15.0
27643,@levelsio,Greece,GR,greece,1994-07-31,1994-07-25,37,23,Athens,athens-greece,6.0


### TRIPS

In [32]:
#get year data
trips_df['year'] = trips_df['date_start'].dt.year
trips_df

,username,country,country_code,country_slug,date_end,date_start,latitude,longitude,place,place_slug,trip_duration,year
0,@lewellenmichael,Mexico,MX,mexico,2018-06-15,2018-06-04,21,-101,Guanajuato,mexico,11.0,2018
1,@lewellenmichael,Mexico,MX,mexico,2018-06-03,2018-05-31,19,-99,Mexico City,mexico-city-mexico,3.0,2018
2,@lewellenmichael,Mexico,MX,mexico,2017-11-05,2017-11-01,21,-86,Cancun,cancun-mexico,4.0,2017
3,@lewellenmichael,Jordan,JO,jordan,2017-08-07,2017-07-24,31,35,Amman,amman-jordan,14.0,2017
4,@waylandchin,China,CN,china,2017-03-18,2017-02-17,40,122,Yingkou,china,29.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...
46505,@antonioc,Cuba,CU,cuba,2016-05-25,2016-05-19,21,-77,Cuba,cuba,6.0,2016
46506,@antonioc,Costa Rica,CR,costa-rica,2016-05-18,2016-04-28,9,-83,Costa Rica,costa-rica,20.0,2016
46507,@antonioc,United States,US,united-states,2016-04-26,2016-04-04,39,-75,Philadelphia,philadelphia-pa-united-states,22.0,2016
46508,@jrearden,United States,US,united-states,2018-05-19,2018-05-12,30,-86,Destin,united-states,7.0,2018


In [33]:
#to populate with country specific data
country_df_data = country_df

In [34]:
country_visits = trips_df.groupby(['country_code'])
#print(country_visits.size())
print(trips_df['country_code'].unique())

['MX' 'JO' 'CN' 'VN' 'HK' 'TH' 'MY' 'KH' 'IN' 'US' 'AE' 'SG' 'UK' 'NL'
 'BE' 'DE' 'CV' 'PL' 'EC' 'CO' 'DK' 'ES' 'ID' 'PT' 'IE' 'AU' 'SE' 'NO'
 'AT' 'SK' 'HU' 'IS' 'NZ' 'CZ' 'CH' 'JP' 'MT' 'IT' 'CA' 'CR' nan 'HR' 'ZA'
 'LU' 'FR' 'AR' 'PH' 'LA' 'TW' 'FI' 'EE' 'BR' 'GR' 'NI' 'BG' 'EG' 'TR'
 'ME' 'GB' 'MA' 'RU' 'LK' 'VA' 'CU' 'RS' 'LB' 'BH' 'QA' 'AW' 'CB' 'KR'
 'NP' 'PE' 'SV' 'GT' 'RO' 'GI' 'CY' 'IL' 'BT' 'UA' 'SI' 'MC' 'TT' 'BO'
 'UY' 'PY' 'TZ' 'OM' 'CL' 'KE' 'UG' 'RW' 'MZ' 'FO' 'PA' 'LT' 'GE' 'MN'
 'PR' 'FJ' 'ET' 'AZ' 'AA' 'AL' 'BA' 'LV' 'JM' 'KG' 'MK' 'VE' 'BZ' 'MG'
 'DO' 'KP' 'IA' 'BD' 'SN' 'SM' 'KS' 'HN' 'SB' 'SA' 'TN' 'SC' 'MV' 'MD'
 'BY' 'PK' 'MU' 'VB' 'BS' 'GD' 'BB' 'KY' 'BW' 'ZW' 'ZM' 'MW' 'LS' 'BI'
 'CT' 'CM' 'GP' 'KZ' 'UZ' 'MM' 'BN' 'RE' 'OI' 'SD' 'LY' 'TJ' 'GH' 'BM'
 'CK' 'TL' 'NG' 'SZ' 'AD' 'GL' 'AF' 'LI' 'IR' 'AO' 'DZ' 'KW' 'GN' 'TO'
 'PS' 'YE' 'DJ' 'SL' 'AI' 'SO' 'NC' 'VG' 'PG']


In [51]:
yearly_visits_by_country = trips_df.groupby(['country_code','year']).size().reset_index('year')
yearly_visits_by_country= yearly_visits_by_country.rename(columns={yearly_visits_by_country.columns[1]: 'no_visits'})

In [58]:
total_visits_by_country = yearly_visits_by_country.groupby('country_code').size().reset_index()
total_visits_by_country= total_visits_by_country.rename(columns={total_visits_by_country.columns[1]: 'no_visits'})
# most visited country 
most_visited_country = total_visits_by_country[total_visits_by_country['no_visits'] == total_visits_by_country['no_visits'].max()]
most_visited_country

#msc_max_followers = msc_df[msc_df['followers'] == msc_df['followers'].max()]

,country_code,no_visits
165,US,33


In [59]:
# least visited country
least_visited_country = total_visits_by_country[total_visits_by_country['no_visits'] == total_visits_by_country['no_visits'].min()]
least_visited_country


,country_code,no_visits
4,AI,1
18,BI,1
37,CT,1
43,DJ,1
46,DZ,1
62,GN,1
66,HK,1
92,LI,1
98,LY,1
114,NC,1


In [54]:
country_df

,region,sub_region
country_code,,
AF,Asia,Southern Asia
AX,Europe,Northern Europe
AL,Europe,Southern Europe
DZ,Africa,Northern Africa
AS,Oceania,Polynesia
...,...,...
WF,Oceania,Polynesia
EH,Africa,Northern Africa
YE,Asia,Western Asia
